The Knapsack Problem.  Can be used to optimize daily tasks if we can estimate the effort and reward(value) for each tasks.  It is bound by the total available time we have.

In [2]:
calls = 0
def value(n,s,m,mem):
  """
  n - tasks left
  s - time left
  m - a list of (value,task)
  mem - cache the intermediate results
  """
  global calls
  calls += 1
  # print(f'#{calls}:n={n},s={s}')
  if n == 0:
    return 0

  if n in mem and s in mem[n]:
    return mem[n][s]

  if(m[n-1][1] > s): #space exceeded
    result = value(n-1, s, m, mem)
  else:
    result = max( m[n-1][0]  + value(n-1, s-m[n-1][1], m, mem ), value(n-1, s, m, mem))
  
  if n not in mem:
    mem[n] = {}
    
  mem[n][s] = result
  return result

In [3]:
def get_opt_tasks(v, t, total_time):
  m = list(zip(v,t))
  efficiency = [i[0]/i[1] for i in m]
  mem = {} #clear cache first
  total = value(len(v), total_time, m, mem)
  # print(f'total recursive calls made={calls}')

  #print out the tasks picked
  i = len(t)  #task
  j = total_time #time
  picked=[]
  while i>0 and j>0:
    # print(f'i={i},j={j}')
    if i-1 in mem and mem[i][j] == mem[i-1][j]:
      # print(f'skip {i}')
      i -= 1
    else:
      # print(f'picked={i},value={v[i-1]},time={t[i-1]},time left={j - t[i-1]}')
      picked.append(i)
      j -= t[i-1] #reduce the effort
      i -=1

  efficiency_picked = [efficiency[i-1] for i in picked]
  tasks_w_eff = zip(picked, efficiency_picked)
  print(f'picked tasks with efficiency={list(tasks_w_eff)}')
  ordered_tasks_index = sorted(range(len(efficiency_picked)), key=lambda k: -efficiency_picked[k])
  ordered_tasks = [picked[i] for i in ordered_tasks_index]
  print(f'ordered tasks={ordered_tasks}')
  print(f'total value={total}')
  return ordered_tasks, total

Example 1

In [4]:
v = (7, 9, 5, 12, 14, 6, 12) #value
t = (3, 4, 2, 6, 7, 3, 5) #task
total_time = 20
get_opt_tasks(v, t, total_time)

picked tasks with efficiency=[(7, 2.4), (4, 2.0), (3, 2.5), (2, 2.25), (1, 2.3333333333333335)]
ordered tasks=[3, 7, 1, 2, 4]
total value=45


([3, 7, 1, 2, 4], 45)

Example 2
Reading the task list from a csv

In [37]:
#upload a excel file to colab
from google.colab import files
data_to_load = files.upload()

#read excel
import io
import pandas as pd
df = pd.read_excel(io.BytesIO(data_to_load['task.xlsx']))
v = df['Reward'].tolist()
t = df['Effort'].tolist()
total_time = 20
get_opt_tasks(v, t, total_time)

picked tasks with efficiency=[(7, 2.4), (4, 2.0), (3, 2.5), (2, 2.25), (1, 2.3333333333333335)]
ordered tasks=[3, 7, 1, 2, 4]
total value=45


([3, 7, 1, 2, 4], 45)

In [ ]:
#read it from google drive
from google.colab import drive
drive.mount('/content/drive')

In [62]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import numpy as np

gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('task').sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:],columns=rows[0]).astype(np.float)
v = df['Reward'].tolist()
t = df['Effort'].tolist()
total_time = 20
get_opt_tasks(v, t, total_time) 

pciked tasks with efficiency=[(7, 2.4), (4, 2.0), (3, 2.5), (2, 2.25), (1, 2.3333333333333335)]
ordered tasks=[3, 7, 1, 2, 4]
total value=45.0


([3, 7, 1, 2, 4], 45.0)